# 19. Ensemble Learning: Stacking

**Purpose:** Learn and revise **Stacking** (stacked generalization) in Scikit-learn.

---

## What is Stacking?

**Stacking** uses **multiple base estimators** and a **final meta-learner** that combines their outputs. Steps:

1. **Base estimators** are fit on the training data.
2. Their **predictions** (or probabilities) on a **hold-out set or via CV** become **features** for the meta-learner.
3. The **meta-learner** (e.g. logistic regression) is fit on these base predictions to produce the final prediction.

Using **cross-validation** to generate base predictions avoids overfitting the meta-learner to the training set. **StackingClassifier** / **StackingRegressor** automate this.

**Key idea:** Meta-learner **learns how to combine** base models; often stronger than simple averaging (voting).

## Concepts to Remember

| Concept | Description |
|--------|-------------|
| **estimators** | List of (name, estimator) base models. |
| **final_estimator** | Meta-learner (e.g. LogisticRegression()); fit on base predictions. |
| **cv** | Folds for generating out-of-fold base predictions; default 5. |
| **When to use** | When you want a learned combination of diverse models; often wins competitions. |

In [1]:
import numpy as np
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
np.random.seed(42)
X = np.random.randn(300, 4)
y = (X[:, 0] * X[:, 1] + X[:, 2] > 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [3]:
estimators = [
    ("lr", Pipeline([("scale", StandardScaler()), ("clf", LogisticRegression(max_iter=1000))])),
    ("tree", DecisionTreeClassifier(max_depth=4)),
    ("svc", Pipeline([("scale", StandardScaler()), ("clf", SVC(probability=True))])),
]
stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), cv=5)
stacking.fit(X_train_s, y_train)
y_pred = stacking.predict(X_test_s)

print("Stacking Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Stacking Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.94      0.97      0.95        31
           1       0.96      0.93      0.95        29

    accuracy                           0.95        60
   macro avg       0.95      0.95      0.95        60
weighted avg       0.95      0.95      0.95        60



## Key Takeaways

- **StackingClassifier** / **StackingRegressor**; **final_estimator** is the meta-learner; **cv** generates out-of-fold base predictions.
- Base estimators should be **diverse**; meta-learner learns optimal weights.
- More expensive than voting (CV loop); use when the gain in accuracy justifies it.